In [1]:
#!/usr/bin/env python3
#coding=utf-8
import time
from Arm_Lib import Arm_Device
import  serial
import threading

ser = serial.Serial('/dev/ttyACM0',9600,timeout=1)
Arm = Arm_Device()
time.sleep(.1)


In [2]:
# Define the function of sandwich block, enable = 1: clamp, = 0: loosen
# 定义夹积木块函数，enable=1：夹住，=0：松开
def arm_clamp_block(enable):
    if enable == 0:
        Arm.Arm_serial_servo_write(6, 60, 1100)
    else:
        Arm.Arm_serial_servo_write(6, 120, 1000)
        time.sleep(0.2)
        Arm.Arm_serial_servo_write(6, 145, 1000)
    time.sleep(.5)

    
# Define the function of mobile manipulator and control the motion of No. 1-5 steering gear at the same time, P = [S1, S2, S3, S4, S5]
# 定义移动机械臂函数,同时控制1-5号舵机运动，p=[S1,S2,S3,S4,S5]
def arm_move(p, s_time = 500):
    for i in range(5):
        id = i + 1
        if id == 5:
            time.sleep(.1)
            Arm.Arm_serial_servo_write(id, p[i], int(s_time*1.2))
        else :
            Arm.Arm_serial_servo_write(id, p[i], s_time)
        time.sleep(.01)
    time.sleep(s_time/1000)

# The manipulator moves upward
# 机械臂向上移动
def arm_move_up():
    Arm.Arm_serial_servo_write(2, 90, 1500)
    Arm.Arm_serial_servo_write(3, 90, 1500)
    Arm.Arm_serial_servo_write(4, 90, 1500)
    time.sleep(.1)


In [3]:
# Define variable parameters at different locations
# 定义不同位置的变量参数红为硬，黄为软
p_mould = [90, 130, 0, 0, 90]
p_top = [90, 80, 50, 50, 270]
p_Brown = [90, 53, 33, 36, 270]

p_Yellow = [65, 22, 64, 56, 270]
p_Red = [117, 19, 66, 56, 270]

p_Green = [136, 66, 20, 29, 270]
p_Blue = [44, 66, 20, 28, 270]


In [4]:
def arm_clamp():#夹取
    arm_clamp_block(0)
    arm_move(p_mould, 1000)
    time.sleep(1)
    
    arm_move(p_top, 1000)
    arm_move(p_Brown, 1000)
    arm_clamp_block(1)
    time.sleep(0.5)
    

In [5]:
# Grab a block from the gray block and put it on the yellow block.
# 从灰色积木块位置抓取一块积木放到黄色积木块的位置上。
def arm_soft():
    
    arm_move(p_top, 1000)
    arm_move(p_Yellow, 1000)
    arm_clamp_block(0)


    arm_move(p_mould, 1000)

    time.sleep(1)


In [6]:
# Grab a block from the gray block and put it on the red block.
# 从灰色积木块位置抓取一块积木放到红色积木块的位置上。
def arm_hard():
    

    arm_move(p_top, 1000)
    arm_move(p_Red, 1000)
    arm_clamp_block(0)

    arm_move_up()
    arm_move(p_mould, 1100)

    time.sleep(1)


In [7]:
#S数据转换
def data_transfrom():
    
    a = ser.read(3)
    print(a,end='\n')
    b = bytes.decode(a)
    value = int(b)
    print(value)
    

In [ ]:
 while True:
        count = ser.inWaiting()
        if count!=0:
            if(b'a'==ser.read()):
                ser.flush()
                arm_clamp()
                
                while True:
                    count = ser.inWaiting()
                    if (count!=0):
                        
                        #S数据转换
                        a = ser.read(3)
                        print(a,end='\n')
                        b = bytes.decode(a)
                        value = int(b)
                        print(value)
                        
                        #软硬判断
                        if(value>540):
                            arm_hard()
                            break
                        else:
                            arm_soft()
                            break
                    
                


b'329'
329
b'654'
654
